In [1]:
!pip install transformers torch torchvision accelerate bitsandbytes datasets pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing in

In [2]:
!pip install --upgrade huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.8/514.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 71.3 MB/s eta 0:00:00
  Attempting uninstall: hf-xet
    Found existing installation: hf-xet 1.1.0
    Uninstalling hf-xet-1.1.0:
      Successfully uninstalled hf-xet-1.1.0
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.31.1
    Uninstalling huggingface-hub-0.31.1:
      Successfully uninstalled huggingface-hub-0.31.1


In [3]:
import os
import pandas as pd

# Base path to dataset
base_path = "/kaggle/input/flickr30k"




# Load and preprocess captions
df=  pd.read_csv("/kaggle/input/flickr30k/captions.txt")
df.head()

,image,caption
0,1000092795.jpg,Two young guys with shaggy hair look at their...
1,1000092795.jpg,"Two young , White males are outside near many..."
2,1000092795.jpg,Two men in green shirts are standing in a yard .
3,1000092795.jpg,A man in a blue shirt standing in a garden .
4,1000092795.jpg,Two friends enjoy time spent together .


In [4]:
import os
import pandas as pd

# Base dataset path
base_path = "/kaggle/input/flickr30k"

# Updated image directory path
image_dir = os.path.join(base_path, "Images", "flickr30k_images")
captions_file = os.path.join(base_path, "captions.txt")

# Load captions
df = pd.read_csv(captions_file)
df.dropna(inplace=True)

# Filter rows where the image file exists
df = df[df["image"].apply(lambda x: os.path.exists(os.path.join(image_dir, x)))]

# Add full image path column only for existing images
df["image_path"] = df["image"].apply(lambda x: os.path.join(image_dir, x))

print(f"Total samples with existing images: {len(df)}")
print(df.head())


Total samples with existing images: 158914
            image                                            caption  \
0  1000092795.jpg   Two young guys with shaggy hair look at their...   
1  1000092795.jpg   Two young , White males are outside near many...   
2  1000092795.jpg   Two men in green shirts are standing in a yard .   
3  1000092795.jpg       A man in a blue shirt standing in a garden .   
4  1000092795.jpg            Two friends enjoy time spent together .   

                                          image_path  
0  /kaggle/input/flickr30k/Images/flickr30k_image...  
1  /kaggle/input/flickr30k/Images/flickr30k_image...  
2  /kaggle/input/flickr30k/Images/flickr30k_image...  
3  /kaggle/input/flickr30k/Images/flickr30k_image...  
4  /kaggle/input/flickr30k/Images/flickr30k_image...  


In [5]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import ViTFeatureExtractor, ViTModel
from transformers import LlamaTokenizer, LlamaForCausalLM
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

2025-06-15 05:35:21.770959: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749965721.973056      18 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749965722.032778      18 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using device: cuda


In [6]:
from sklearn.model_selection import train_test_split

# Assuming train_df is your original dataset
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [7]:
print("Total samples:", len(train_df))
print(train_df.head())

Total samples: 127131
                 image                                            caption  \
50762   2974798723.jpg   A skateboarder is attempting to jump down a s...   
127083  4943870853.jpg   Three people sitting on the curb in an Asian ...   
125863  4928300049.jpg   An outdoor event with a guy securing a tarp t...   
106224  4604410267.jpg    A woman on her cellphone is walking in a park .   
20789   2219959872.jpg           A man poses outside a tourist location .   

                                               image_path  
50762   /kaggle/input/flickr30k/Images/flickr30k_image...  
127083  /kaggle/input/flickr30k/Images/flickr30k_image...  
125863  /kaggle/input/flickr30k/Images/flickr30k_image...  
106224  /kaggle/input/flickr30k/Images/flickr30k_image...  
20789   /kaggle/input/flickr30k/Images/flickr30k_image...  


In [8]:
print("Total samples:", len(val_df))
print(val_df.head())

Total samples: 31783
                 image                                            caption  \
138302  5797756884.jpg   A man is sitting on an upturned white bucket ...   
1325     109260218.jpg   Spelunkers pose inside a rock cavern while ba...   
61070   3218861747.jpg   A skateboarder is doing a trick in the air wi...   
113574  4726678447.jpg   Man with backpack walks through a crowded park .   
22388    226481576.jpg   An attractive young New York City policewoman...   

                                               image_path  
138302  /kaggle/input/flickr30k/Images/flickr30k_image...  
1325    /kaggle/input/flickr30k/Images/flickr30k_image...  
61070   /kaggle/input/flickr30k/Images/flickr30k_image...  
113574  /kaggle/input/flickr30k/Images/flickr30k_image...  
22388   /kaggle/input/flickr30k/Images/flickr30k_image...  


In [9]:


def create_dataframe(dff):
    data = []
    i = 0
    for _, item in dff.iterrows():
        
        image = item['image_path']  # Load and convert image
        row = {
            'image_path': image,
            'caption': item['caption']
        }
        data.append(row)
    return pd.DataFrame(data)

train_df = create_dataframe(train_df)
val_df = create_dataframe(val_df)


In [10]:
train_df.shape

(127131, 2)

In [11]:
val_df.shape

(31783, 2)

In [12]:
from PIL import Image
import torch
from torch.utils.data import Dataset

class Flickr8kDataset(Dataset):
    def __init__(self, df, feature_extractor, tokenizer, max_length=128):
        self.df = df
        self.feature_extractor = feature_extractor
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row['image_path']).convert('RGB')
        caption = row['caption']

        # Extract pixel values for the image
        pixel_values = self.feature_extractor(image, return_tensors="pt").pixel_values.squeeze(0)

        # Tokenize caption
        tokenized_caption = self.tokenizer(
            caption,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        input_ids = tokenized_caption.input_ids.squeeze()

        # Replace pad token with -100 so it's ignored in loss computation
        input_ids[input_ids == self.tokenizer.pad_token_id] = -100

        return {
            "pixel_values": pixel_values,
            "labels": input_ids
        }


In [13]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    ViTModel,
    BartForConditionalGeneration,
    BartTokenizer,
    VisionEncoderDecoderModel,
    ViTImageProcessor,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    BartConfig,
    BartForCausalLM
)
from torch.nn.utils.rnn import pad_sequence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load encoder and config for decoder
encoder = ViTModel.from_pretrained("google/vit-base-patch16-224")

# Create Bart decoder config with decoder settings
decoder_config = BartConfig.from_pretrained("facebook/bart-base")
decoder_config.is_decoder = True
decoder_config.add_cross_attention = True

# Initialize decoder using the config
decoder = BartForCausalLM(config=decoder_config)

# Now create VisionEncoderDecoderModel
model = VisionEncoderDecoderModel(encoder=encoder, decoder=decoder)
model.to(device)

# Load tokenizer and image processor
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
feature_extractor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")

# Set special tokens and config attributes
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = tokenizer.vocab_size
model.decoder.resize_token_embeddings(tokenizer.vocab_size)
model.tie_weights()

# Optional generation config
"""
This happens because generation_max_length=128 and other parameters 
are passed in TrainingArguments, but the model's internal GenerationConfig 
does not have them set yet.

This is not harmful — it just warns that the model will use these values 
from now on in the generation_config.

✅ Fix (Optional):
If you want to eliminate this warning permanently, 
set these values explicitly in model.generation_config before training.
"""

model.generation_config.max_length = 128
model.generation_config.early_stopping = True
model.generation_config.num_beams = 6

# Data collator for batching
def collate_fn(batch):
    pixel_values = torch.stack([item["pixel_values"] for item in batch])
    labels = [item["labels"] for item in batch]
    labels = pad_sequence(labels, batch_first=True, padding_value=-100)
    return {"pixel_values": pixel_values, "labels": labels}

# Assuming train_df and val_df are already loaded
# train_subset_df = train_df.sample(n=100, random_state=42).reset_index(drop=True)
# val_subset_df = val_df.sample(n=50, random_state=42).reset_index(drop=True)

train_dataset = Flickr8kDataset(train_df, feature_extractor, tokenizer)
val_dataset = Flickr8kDataset(val_df, feature_extractor, tokenizer)

# Dataloader settings
cpu_count = os.cpu_count()
num_workers = min(4, cpu_count // 2) if cpu_count else 2
pin_memory = torch.cuda.is_available()

train_dataloader = DataLoader(
    train_dataset,
    batch_size=34,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=num_workers,
    pin_memory=pin_memory
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=34,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=num_workers,
    pin_memory=pin_memory
)

print(f"DataLoaders created with num_workers={num_workers}, pin_memory={pin_memory}")


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

DataLoaders created with num_workers=2, pin_memory=True


In [14]:
!pip install evaluate
!pip install rouge_score



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=0f6a22c2f36f433c7b6519599320d362801aef19c93019aacfbbf46e82db1d98
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [15]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from evaluate import load
import numpy as np
import torch

# Load ROUGE metric
rouge = load("rouge")

# Compute metrics function
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # Replace -100 in labels with tokenizer.pad_token_id
    if isinstance(labels, tuple):
        labels = labels[0]

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Ensure lists of lists with int values
    preds = preds.tolist()
    labels = labels.tolist()

    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Clean text
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    # Prepare labels for ROUGE
    decoded_labels = [[label] for label in decoded_labels]

    # Compute ROUGE
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    return {
        "rouge1": result["rouge1"],
        "rouge2": result["rouge2"],
        "rougeL": result["rougeL"]
    }


# Custom Trainer class for Vision-Text model
class VisionSeq2SeqTrainer(Seq2SeqTrainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        pixel_values = inputs["pixel_values"]
        labels = inputs["labels"]
        outputs = model(pixel_values=pixel_values, labels=labels)
        loss = outputs.loss
        return (loss, outputs) if return_outputs else loss

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./vit-bart-captioning",
    per_device_train_batch_size=34,
    per_device_eval_batch_size=34,
    predict_with_generate=True,
    generation_max_length=128,
    generation_num_beams=6,

    # Logging & evaluation
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    # Optimization & performance
    num_train_epochs=5,
    learning_rate=1e-4,
    warmup_steps=100,
    fp16=torch.cuda.is_available(),
    gradient_checkpointing=True,
    remove_unused_columns=False,
    dataloader_pin_memory=True,
    dataloader_num_workers=2,

    # Misc
    report_to="none",
    disable_tqdm=False
)

# Instantiate trainer
trainer = VisionSeq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,           # Optional but helpful for decoding
    data_collator=collate_fn,
    compute_metrics=compute_metrics
)

# Start training
trainer.train()


/tmp/ipykernel_18/670199431.py:88: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `VisionSeq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = VisionSeq2SeqTrainer(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel
1,4.196300,3.672144,0.303255,0.010360,0.256429
2,3.412100,3.420340,0.303031,0.011656,0.262388
3,3.066700,3.319830,0.315307,0.010758,0.268475
4,2.792600,3.287727,0.302119,0.010187,0.261697
5,2.569100,3.303226,0.309363,0.010796,0.265431


There were missing keys in the checkpoint model loaded: ['decoder.lm_head.weight'].


TrainOutput(global_step=18700, training_loss=3.207367041276738, metrics={'train_runtime': 35129.3134, 'train_samples_per_second': 18.095, 'train_steps_per_second': 0.532, 'total_flos': 8.215500250491126e+19, 'train_loss': 3.207367041276738, 'epoch': 5.0})

In [16]:
# Set save path
save_dir = "./vit_bart_captioning_model"

# Save everything
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
feature_extractor.save_pretrained(save_dir)

print(f"Model, tokenizer, and feature extractor saved to {save_dir}")


Model, tokenizer, and feature extractor saved to ./vit_bart_captioning_model


In [17]:
'''
from transformers import VisionEncoderDecoderModel, BartTokenizer, ViTImageProcessor
from PIL import Image
import torch

# Load everything
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VisionEncoderDecoderModel.from_pretrained("./vit_bart_captioning_model").to(device)
tokenizer = BartTokenizer.from_pretrained("./vit_bart_captioning_model")
feature_extractor = ViTImageProcessor.from_pretrained("./vit_bart_captioning_model")

# Ensure decoding settings are restored
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.generation_config.max_length = 128
model.generation_config.num_beams = 4

# Wrapper function to generate caption
def generate_caption(image_path):
    image = Image.open(image_path).convert("RGB")
    pixel_values = feature_extractor(images=image, return_tensors="pt").pixel_values.to(device)

    with torch.no_grad():
        output_ids = model.generate(pixel_values)
    
    caption = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return caption.strip()

# Example usage
# print(generate_caption("path_to_image.jpg"))

'''

'\nfrom transformers import VisionEncoderDecoderModel, BartTokenizer, ViTImageProcessor\nfrom PIL import Image\nimport torch\n\n# Load everything\ndevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")\nmodel = VisionEncoderDecoderModel.from_pretrained("./vit_bart_captioning_model").to(device)\ntokenizer = BartTokenizer.from_pretrained("./vit_bart_captioning_model")\nfeature_extractor = ViTImageProcessor.from_pretrained("./vit_bart_captioning_model")\n\n# Ensure decoding settings are restored\nmodel.config.decoder_start_token_id = tokenizer.bos_token_id\nmodel.config.eos_token_id = tokenizer.eos_token_id\nmodel.config.pad_token_id = tokenizer.pad_token_id\nmodel.generation_config.max_length = 128\nmodel.generation_config.num_beams = 4\n\n# Wrapper function to generate caption\ndef generate_caption(image_path):\n    image = Image.open(image_path).convert("RGB")\n    pixel_values = feature_extractor(images=image, return_tensors="pt").pixel_values.to(device)\n\n    with tor